In [2]:
import os
import datetime
import pandas as pd
import re
import agriculture as ag

ModuleNotFoundError: No module named 'agriculture'

In [2]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Producto: Acelga - Primera
Producto: Aguacate Criollo - Primera
Producto: Aguacate Fuerte - Primera
Producto: Aguacate Hass - Calidad extra
Producto: Aguacate Hass - Calidad super extra
Producto: Aguacate Hass - Primera
Producto: Aguacate Hass adelantado - Primera
Producto: Aguacate Hass flor vieja - Primera
Producto: Aguacate Pagua - Primera
Producto: Ajo Blanco - Primera
Producto: Ajo Blanco # 8 - Primera
Producto: Ajo Morado - Primera
Producto: Apio - Primera
Producto: Berenjena - Primera
Producto: Betabel - Primera
Producto: Brócoli - Primera
Producto: Cacahuate - Primera
Producto: Calabacita Criolla - Primera
Producto: Calabacita Italiana - Primera
Producto: Calabacita regional - Primera
Producto: Calabaza de castilla - Primera
Producto: Camote - Primera
Producto: Caña - Primera
Producto: Cebolla Bola - Primera
Producto: Cebolla Bola grande - Primera
Producto: Cebolla de rabo - Primera
Producto: Cebolla Morada - Primera
Producto: Cilantro - Primera
Producto: Ciruela Amarilla - Pri

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [4]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = "./data/sniim/"

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [5]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [6]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [7]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [8]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
235608,15/07/2021,Cebolla De Rabo,Manojo,Durango,Durango,"Central de Abasto ""Francisco Villa""",6.0,7.0,7.00,NaN
1682648,10/03/2023,Toronja Roja,Kilogramo,Baja California,Baja California Sur,Unión de Comerciantes de La Paz,15.0,28.0,28.00,NaN
783663,03/09/2020,Cacahuate,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,28.0,30.0,29.00,NaN
1203415,09/11/2022,Limon Con Semilla #5,Caja de 15 kg.,Colima,Durango,Centro de Distribución y Abasto de Gómez Palacio,20.0,22.0,21.00,NaN
228133,07/03/2022,Chile Serrano,Arpilla de 30 kg.,San Luis Potosí,México,Central de Abasto de Toluca,23.0,26.0,23.00,NaN
2008838,29/03/2021,Mango Ataulfo,Caja de 30 kg.,Chiapas,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,12.0,12.67,12.33,NaN
1794558,21/04/2020,Pera D´Anjou #100,Caja de 18 kg.,Importación,Zacatecas,Mercado de Abasto de Zacatecas,28.89,28.89,28.89,NaN
235257,27/07/2021,Cebolla De Rabo,Manojo,Coahuila,Chihuahua,Mercado de Abasto de Cd. Juárez,16.0,16.0,16.00,NaN
827698,05/07/2021,Nopal Grande,Ciento,Distrito Federal,México,Central de Abasto de Toluca,0.8,1.0,0.90,NaN
89916,13/10/2020,Chile Guajillo,Kilogramo,Zacatecas,Morelos,Central de Abasto de Cuautla,80.0,82.0,80.00,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [9]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [10]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df

,0
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...
obs,Observaciones encontradas para la muestra


## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [11]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 96.03%


Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [12]:
precios_sniim.drop(columns="obs", inplace=True)

In [13]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
422022,28/12/2022,Col Mediana,Docena,Puebla,México,Central de Abasto de Toluca,17.0,19.0,17.00
903058,17/05/2023,Chile Puya Seco,Kilogramo,Zacatecas,Querétaro,Mercado de Abasto de Querétaro,110.0,115.0,110.00
136121,18/01/2023,Rabano,Rollo de 5 kg.,Puebla,Veracruz,Mercado Malibrán,4.0,6.4,5.00
301753,29/05/2023,Jamaica,Kilogramo,Puebla,Veracruz,Mercado Malibrán,190.0,199.0,195.00
53012,18/02/2021,Chile Poblano,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,21.0,23.0,22.00
173126,23/03/2022,Piña Grande,Pieza,Veracruz,DF,Central de Abasto de Iztapalapa DF,30.0,34.0,32.00
250580,10/05/2021,Chile Hungaro,Kilogramo,Zacatecas,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,16.0,18.0,17.00
225803,12/12/2022,Chile Serrano,Arpilla de 20 kg.,Sinaloa,Chihuahua,Central de Abasto de Chihuahua,22.75,38.0,22.75
1345520,10/03/2022,Platano Macho,Kilogramo,Veracruz,Tamaulipas,"Módulo de Abasto de Tampico, Madero y Altamira",37.0,41.0,41.00
696674,11/09/2020,Platano Dominico,Caja de 18 kg.,Jalisco,Baja California Sur,Unión de Comerciantes de La Paz,12.78,21.11,12.78


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [14]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   fecha         object 
 1   producto      object 
 2   presentacion  object 
 3   origen        object 
 4   destino       object 
 5   central       object 
 6   precio_min    object 
 7   precio_max    object 
 8   precio_frec   float64
dtypes: float64(1), object(8)
memory usage: 138.7+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [15]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [16]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'México', 'Quintana Roo', 'Sinaloa', 'Puebla',
       'Nuevo León', 'Aguascalientes', 'Zacatecas', 'Tamaulipas',
       'Chihuahua', 'Coahuila', 'Guerrero', 'Colima', 'Veracruz',
       'Querétaro', 'Tabasco', 'Morelos', 'Baja California Sur',
       'Nacional', 'Sin Especificar'], dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [17]:
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California ', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [18]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [19]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

In [20]:
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(str).str.replace(",","")
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [21]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
280410,2023-08-30,Chayote Sin Espinas,Caja de 18 kg.,Veracruz,México,Central de Abasto de Toluca,11.11,12.22,12.22
1181256,2021-07-29,Papaya Maradol,Kilogramo,Chiapas,Chiapas,Central de Abasto de Tuxtla Gutiérrez,11.00,13.00,11.00
578492,2020-01-29,Calabacita Criolla,Kilogramo,Jalisco,Chihuahua,Mercado de Abasto de Cd. Juárez,16.00,16.00,16.00
1626533,2022-07-18,Nopal,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,31.00,33.00,32.00
1757153,2020-08-21,Aguacate Criollo,Caja de 10 kg.,Nuevo León,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",16.00,20.00,18.00
116512,2022-03-28,Zanahoria Polvo,Arpilla de 23 kg.,Guanajuato,Jalisco,Mercado de Abasto de Guadalajara,3.48,3.70,3.48
549554,2020-05-22,Fresa,Caja de 9 kg.,Importación,Durango,Centro de Distribución y Abasto de Gómez Palacio,35.56,36.11,35.89
521175,2023-08-23,Cebolla Bola,Kilogramo,Guanajuato,Coahuila,"Central de Abasto de La Laguna, Torreón",14.00,18.00,18.00
1338246,2022-03-05,Melon Cantaloupe Sin Clasificacion,Kilogramo,Coahuila,Querétaro,Mercado de Abasto de Querétaro,10.00,11.00,10.00
444464,2023-02-28,Fresa,Caja de 5 kg.,Importación,Jalisco,Mercado Felipe Ángeles de Guadalajara,80.00,80.00,80.00


In [22]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ MB
